In [44]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier

In [45]:
def preparexy(df):
    # Data Preparing
    # Encode labels with value 0-> n_classes -1
    le = preprocessing.LabelEncoder()
    features = ['I50', 'Φ50', 'I150', 'Φ150', 'I250', 'Φ250']
    
    # Features - X
    X = df[features]

    # Calculate Z-score in order to find outliers
    z = np.abs(stats.zscore(X))
    #print(z) # Visualize
    threshold = 2.5 # Change the threshold arbitrarily
    #print(np.where(z > threshold))
    df = df[(z < threshold).all(axis=1)] # Remove outliers that exceed the threshold given from dataset

    # Now get as X the 'clean' features
    X = df[features]

    # Scale the inputs (4 options; put comment in the ones you are not using)

    # -1- Standard Scaling
    #scaler = StandardScaler()
    #X = scaler.fit_transform(X)

    # -2- Min-max scaling 
    #scaler = preprocessing.MinMaxScaler()
    #X = scaler.fit_transform(X)

    # -3- Robust scaling
    scaler = preprocessing.RobustScaler()
    X = scaler.fit_transform(X)

    # -4- Normalizing
    #scaler = preprocessing.Normalizer()
    #X = scaler.fit_transform(X)

    # Label - Y
    y = df['appliance']

    # Convert y to integer 
    le.fit(y) # Fit label encoder
    y = le.transform(y) # Transform labels to normalized encoding
    
    return(X,y)

def data_process(dataset_path):
    df = pd.read_excel(dataset_path)

    # Data Cleaning with Aggelos Rules for 50, 150, 250 phases
    df = df[(df.I50 > 0.1) & (df.I150 > 0.01) & (df.I250 > 0.01)] # Clean useless current features


    #df['Φ50'] = df['Φ50'].apply(pd.to_numeric)

    # For angle between (90, 180):
    # Modify by +180 degrees
    rows_with_rads_to_decrease = df.loc[(df['Φ50'] > 90) & (df['Φ50'] < 180)]
    rows_with_rads_to_decrease['Φ50'] -= 180
    df.update(rows_with_rads_to_decrease)

    # For angle between (-180, -90):
    # Modify by -180 degrees
    rows_with_rads_to_increase = (df.loc[(df['Φ50'] < -90) & (df['Φ50'] > -180)])
    rows_with_rads_to_increase['Φ50'] += 180
    df.update(rows_with_rads_to_increase)
    
    return df

X_train, y_train = preparexy(data_process("../datasets/appliances_combination_daskio.xls"))
X_test, y_test = preparexy(data_process("../datasets/appliances_combination_veroia.xls"))

/home/klajdi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/klajdi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [47]:
# Decision Trees
clf = DecisionTreeClassifier(random_state = 42) # Feel free to change 'min_samples_split' 
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.2521719250114312